In [1]:
weather_data = sc.textFile("/FileStore/tables/pm2_5Taiwan-3b69f.csv")

In [2]:
weather_data_rdd = weather_data.map(lambda line : line.split(","))
for x in weather_data_rdd.take(30):
    for i in range(len(x)):
        print x[i],
    print ""

In [3]:
pm25schema = weather_data_rdd.first()
print pm25schema

In [4]:
import math
def remove_row_with_noise (x):
    for i in range(3, len(x)):
        if not x[i].isdecimal():
            return False
    return True 

In [5]:
clean_weather_data = weather_data_rdd\
                    .filter(lambda x: x!=pm25schema)\
                    .filter(remove_row_with_noise)
clean_weather_data.cache()

In [6]:
dalipm25 = clean_weather_data.filter(lambda x: x[1] == u'大里')

In [7]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
dalipm25row = dalipm25.map(lambda p:
        Row(
        date = p[0],
        location = p[1],
        measure = p[2],
        hr_01 = float(p[3]), hr_02 = float(p[4]),hr_03 = float(p[5]),hr_04 = float(p[6]),hr_05 = float(p[7]),
        hr_06 = float(p[8]),hr_07 = float(p[9]),hr_08 = float(p[10]),hr_09 = float(p[11]),hr_10 = float(p[12]),
        hr_11 = float(p[13]),hr_12 = float(p[14]),hr_13 = float(p[15]),hr_14 = float(p[16]),hr_15 = float(p[17]),
        hr_16 = float(p[18]),hr_17 = float(p[19]),hr_18 = float(p[20]),hr_19 = float(p[21]),hr_20 = float(p[22]),
        hr_21 = float(p[23]),hr_22 = float(p[24]),hr_23 = float(p[25]),hr_24 = float(p[26]),
    )
)


dalipm25row.take(5)

In [8]:
df = sqlContext.createDataFrame(dalipm25row)

In [9]:
df.show()

In [10]:
df_pm10 = df.select(df.date.alias("datepm10"),df.hr_09.alias("hr_09_pm10"),df.hr_10.alias("hr_10_pm10"), df.hr_11.alias("hr_11_pm10"),df.hr_12.alias("hr_12_pm10"), "measure").filter(df.measure=="PM10")
df_pm25 = df.select(df.date.alias("datepm25"),df.hr_09.alias("hr_09_pm25"),df.hr_10.alias("hr_10_pm25"), df.hr_11.alias("hr_11_pm25"),df.hr_12.alias("hr_12_pm25"), "measure").filter(df.measure=="PM2.5") 
df_AMB_TEMP = df.select(df.date.alias("dateAMB_TEMP"),df.hr_09.alias("hr_09_AMB_TEMP"),df.hr_10.alias("hr_10_AMB_TEMP"), df.hr_11.alias("hr_11_AMB_TEMP"),df.hr_12.alias("hr_12_AMB_TEMP"), "measure").filter(df.measure=="AMB_TEMP")

In [11]:
traing_data = df_pm25\
    .join(df_pm10, df_pm25.datepm25==df_pm10.datepm10)\
    .join(df_AMB_TEMP, df_pm25.datepm25==df_AMB_TEMP.dateAMB_TEMP).drop("dateAMB_TEMP").drop("measure").drop("datepm10")

In [12]:
traing_data.show()

In [13]:
traing_data.corr("hr_09_pm25", "hr_10_pm25")

In [14]:
traing_data.show()

In [15]:
from pyspark.sql import functions as F
formulated_traning_data = traing_data.select("*", F.when(traing_data.hr_12_pm25 > 50, 1).otherwise(0)).withColumnRenamed("CASE WHEN (hr_12_pm25 > 50) THEN 1 ELSE 0 END", "pm25_condiction").\
drop("hr_12_AMB_TEMP").drop("hr_12_pm10").drop("hr_12_pm25").drop("datepm25")

In [16]:
formulated_traning_data.show()

In [17]:
from pyspark.mllib.regression import LabeledPoint

LabelPoints = formulated_traning_data.rdd\
            .map(lambda r: LabeledPoint(r.pm25_condiction, \
                                        [r.hr_09_pm25, r.hr_10_pm25, r.hr_11_pm25, \
                                         r.hr_09_pm10, r.hr_10_pm10, r.hr_11_pm10, \
                                         r.hr_09_AMB_TEMP, r.hr_10_AMB_TEMP, r.hr_11_AMB_TEMP]))


In [18]:
(trainData,validationData,testData) = LabelPoints.randomSplit([6,0,4])

In [19]:
from pyspark.mllib.tree import DecisionTree
DTModel = DecisionTree.trainClassifier(trainData,
        numClasses=2,
        categoricalFeaturesInfo={},
        impurity="entropy",
        maxDepth=3,
        maxBins=5)

In [20]:
prediction = DTModel.predict(testData.map(lambda x: x.features))

In [21]:
predictionAndLabels = prediction.zip(testData.map(lambda x: x.label))

In [22]:
predictionAndLabels.collect()

In [23]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
metrics = BinaryClassificationMetrics(predictionAndLabels)

In [24]:
print metrics.areaUnderROC